In [ ]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
#library(mediation)
library(caret)
#library(gbm)
#library(party)
#library(Metrics)
#install.packages("wesanderson")
#library(wesanderson)
#library(gtools)
#library(interactions)
#install.packages('ggstance')
#library(ggstance)
#install.packages('elasticnet')
#library(elasticnet)
#install.packages('kernlab')
#library(kernlab)
#install.packages('e1071')
#library(e1071)
#install.packages('MatchIt')
#library(MatchIt)
library(dplyr)
#library(ukbtools)
#library(mediation)
library(lavaan)
#install.packages('MissMech')
#library(semTools)
#library(MissMech)
#install.packages('fsbrain')
#library(fsbrain)
#install.packages('ipw')
#library(ipw)
#install.packages('episensr')
#install.packages('measurements')
#library(measurements)
#library(tidyr)
#library(optimx)
#install.packages('devtools')
#library(devtools)
#library(reshape)
library(childsds)
library(stringr)

## Read the data

In [ ]:
ABCD_longitudinal=read.table('/dagher/dagher11/filip/PRSOB/data/ABCD_base_fup1_wide.csv',sep=',',dec='.', header=T)
ABCD=read.table('/dagher/dagher11/filip/PRSOB/data/ABCD_base_fup1_long.csv',sep=',',dec='.', header=T)

In [ ]:
ABCD_longitudinal$deltaVol=ABCD_longitudinal$H_site_smri_vol_cdk_total.y-ABCD_longitudinal$H_site_smri_vol_cdk_total.x
ABCD_longitudinal$deltaThick=ABCD_longitudinal$H_site_smri_thick_cdk_mean.y-ABCD_longitudinal$H_site_smri_thick_cdk_mean.x
ABCD_longitudinal$deltaSA=ABCD_longitudinal$H_site_smri_area_cdk_total.y-ABCD_longitudinal$H_site_smri_area_cdk_total.x

In [ ]:
ABCD=merge(ABCD, dplyr::select(ABCD_longitudinal, subjectkey, deltaBMI_SD, deltaVol, deltaThick, deltaSA), all.x=T)

In [ ]:
PCs=read.table('/dagher/dagher11/filip/Downloads/ABCD_pre-impute.eigenvec', header=T)

## Remove duplicates

In [ ]:
ABCD=ABCD[!duplicated(ABCD$subjectkey),]

In [ ]:
ABCD=subset(ABCD, ABCD$demo_race_a_p___10==1)

## Remove all participants who failed WM QC for all sample consistency

In [ ]:
ABCD=subset(ABCD, ABCD$dmri_dti_postqc_qc==1 | is.na(ABCD$dmri_dti_postqc_qc))

## Read definitions

In [ ]:
aliases=read.table('/dagher/dagher11/filip/ABCD_data/abcd_smrip101_definitions.csv',quote='"', header=T, sep=',')
aliases2=read.table('/dagher/dagher11/filip/ABCD_data/abcd_smrip201_definitions.csv',quote='"', header=T, sep=',')

In [ ]:
ABCD=ABCD[complete.cases(ABCD$H_dmri_dtifa_fiberat_fxrh),]

In [ ]:
new_PRS=read.table('/dagher/dagher11/filip/Downloads/BMI_PRS_new.csv', header=T, sep=',')
colnames(new_PRS)=c( 'X', 'FID' ,'IID' ,'PRS', 'Z_UKB')

In [ ]:
ABCD=merge(ABCD, PCs, by.x='subjectkey', by.y='IID', all.x=T)
ABCD=merge(ABCD, new_PRS, by.x='subjectkey', by.y='IID', all.x=T)

## Read in ADI data

In [ ]:
data_list=list()

files=c('abcd_rhds01.txt')

for (i in 1:length(files)){
    data = readLines(paste("/dagher/dagher9/filip/ABCD/Package_1203460/",files[i],sep=''))
    skip_second = data[-2]
    data = read.csv(textConnection(skip_second), header = TRUE, stringsAsFactors = FALSE, sep='\t')
    drops <- c("dataset_id", "interview_date",'collection_id','src_subject_id')
    data = data[ , !(names(data) %in% drops)]    
    data_list[[i]]=data
}



In [ ]:
data_list[[1]]=data_list[[1]][data_list[[1]]$eventname=='baseline_year_1_arm_1',]

In [ ]:
ABCD=merge((ABCD), dplyr::select(data_list[[1]], subjectkey, reshist_addr1_coi_z_ed_nat,
                             reshist_addr1_coi_z_se_nat,
                             reshist_addr1_coi_z_he_nat), by='subjectkey', all.x=T)

## Correct PRS for PCs

In [ ]:
ABCD$Z_corr=resid(lm(Z ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, 
                     data=ABCD, na.action=na.exclude))

ABCD$Z_corr_UKB=resid(lm(Z_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, 
                     data=ABCD, na.action=na.exclude))

In [ ]:
nrow(ABCD)

## Descriptive statistics of the sample

In [ ]:
describe(ABCD$age_yrs)
describe(ABCD$BMI)
describe(ABCD$BMI_SD)
describe(ABCD$weight_kg)
describe(ABCD$weight_SD)
table(ABCD$sex)

## BMI-PRS and BMI SD corelation

In [ ]:
summary((lm(ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
           PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat +
           reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat +
           interview_age + interview_age + interview_age*sex , data=ABCD)))

summary(lm(ABCD$BMI_SD ~ Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
           PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat +
           reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat +
           interview_age + interview_age + interview_age*sex, data=ABCD))

# Figure 1

In [ ]:
std_mean <- function(x) sd(x)/sqrt(length(x))

ABCD_sum = ABCD %>%  group_by(group = cut(Z_UKB, breaks = seq(-4.5, 4.5, 0.5))) %>%
  summarise(Num = n(), mean = (mean(BMI_SD,na.rm=TRUE)), se=std_mean(BMI_SD))



In [ ]:
col_graph='cadetblue'

#tiff('/dagher/dagher11/filip/PRSOB/Figures/BMI_SDS_plot.tiff', 
#     width=15, height=15, units='cm', res=300)
fig1a=ggplot() +
geom_bar(mapping=aes(x=ABCD_sum$group, y=ABCD_sum$Num), stat='identity', fill=col_graph, color=col_graph) +
geom_point(mapping=aes(x=ABCD_sum$group, y=ABCD_sum$mean*400)) +
geom_errorbar(data=ABCD_sum, mapping=aes(x=group, y=mean*400, ymin=mean*400-se*400, ymax=mean*400+se*400), 
              width=.6,position=position_dodge(.9)) +
geom_smooth(data=ABCD_sum, mapping=aes(x=as.numeric(group), y=mean*400), method='lm', color='black', se=F, 
            size=0.5) +
scale_y_continuous(name='Number of individuals', breaks=c(0, 100, 200, 300, 400, 500, 600, 700, 800), 
                   sec.axis = sec_axis(~./300, name = "BMI SDS", 
                                       breaks=c(-2.5, -2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5))) +
theme_minimal() +
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=12), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
scale_x_discrete(name='BMI-PRS z-score') 

#dev.off()
  #geom_point(aes(y=BMI_SD))
  #scale_y_continuous(name="cyl", sec.axis=sec_axis(~./scaleFactor, name="hp"))

ABCD_plot1b=select(ABCD, BMI_SD, site_id_l)

ABCD_plot1b=ABCD_plot1b %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(BMI_SD),
    sd = sd(BMI_SD),
    n = n(),
    se = sd / n
  ) 

  fig1b=ggplot(data=ABCD_plot1b, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('BMI SDS') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 

ABCD_plot1c=select(ABCD, Z_UKB, site_id_l)

ABCD_plot1c=ABCD_plot1c %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(Z_UKB),
    sd = sd(Z_UKB),
    n = n(),
    se = sd / n
  )

  fig1c=ggplot(data=ABCD_plot1c, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('BMI-PRS') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 


ABCD_plot1d=select(ABCD, PC1, site_id_l)

ABCD_plot1d=ABCD_plot1d %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(PC1),
    sd = sd(PC1),
    n = n(),
    se = sd / n
  )

  fig1d=ggplot(data=ABCD_plot1d, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('Genetic principal component 1') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 


In [ ]:
library('ggpubr')

tiff('/dagher/dagher11/filip/PRSOB/Figures/Figure1.tiff', 
     width=30, height=30, units='cm', res=300)

figure1 <- ggarrange(fig1a, fig1b, fig1c, fig1d,
                    labels = c("a", "b", "c", "d"),
                    ncol = 2, nrow = 2)
figure1

dev.off()


# SEM

In [ ]:
cog=read.table('/dagher/dagher11/filip/PRSOB/data/abcd_execfunc.csv', header=T, sep=',', dec='.')
colnames(cog)=c('subjectkey',colnames(cog)[2:ncol(cog)])
ABCD=merge(ABCD, cog, by='subjectkey',all.x=T)

In [ ]:
hdr <- read.table('/export01/data/fmorys/ABCD30/abcd_mhy02.txt', sep='\t',header=T, quote='"', stringsAsFactors = FALSE)
inh3 <- read.table('/export01/data/fmorys/ABCD30/abcd_mhy02.txt', sep='\t',header=F, quote='"', skip = 2)
colnames( inh3 ) <- colnames(hdr)
inh3=subset(inh3, select=-c(sex, collection_title, study_cohort_name, dataset_id, interview_date, 
                          collection_id, src_subject_id, interview_age)) #Remove unnecessary columns for quicker merging further on
inh3=inh3[inh3$eventname=='baseline_year_1_arm_1',]

ABCD=merge(ABCD, inh3, by='subjectkey',all.x=T)

ABCD$H_WM_FA=rowMeans(data.frame(ABCD$H_site_dmri_dtifa_fiberat_allfiblh, ABCD$H_site_dmri_dtifa_fiberat_allfibrh))


In [ ]:
ABCD$H_site_smri_vol_cdk_total_PC=resid(lm(ABCD$H_site_smri_vol_cdk_total ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$H_site_smri_thick_cdk_mean_PC=resid(lm(ABCD$H_site_smri_thick_cdk_mean ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$H_WM_FA_PC=resid(lm(ABCD$H_WM_FA ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$BMI_SD_site=resid(lm(ABCD$BMI_SD ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$deltaBMI_SD_site=resid(lm(ABCD$deltaBMI_SD ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + BMI_SD_site, na.action=na.exclude, data=ABCD))
ABCD$z.scores_site=resid(lm(ABCD$z.scores ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$Z_corr_site=resid(lm(ABCD$Z_corr ~ ABCD$site_id_l, na.action=na.exclude))
ABCD$Z_corr_UKB_site=resid(lm(ABCD$Z_corr_UKB ~ ABCD$site_id_l, na.action=na.exclude))
ABCD$upps_y_ss_positive_urgency_site=resid(lm(ABCD$upps_y_ss_positive_urgency ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$upps_y_ss_negative_urgency_site=resid(lm(ABCD$upps_y_ss_negative_urgency ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))

In [ ]:
Lavaan_SEM=dplyr::select(ABCD, 
                         H_site_smri_vol_cdk_total_PC,
                        H_site_smri_thick_cdk_mean_PC,
                        H_WM_FA_PC,
                        Z_corr_UKB_site,
                        z.scores_site,
                        upps_y_ss_positive_urgency_site,
                        upps_y_ss_negative_urgency_site,
                        BMI_SD_site,
                        deltaBMI_SD_site,
                        demo_comb_income_v2,
                        demo_ed_v2,
                        demo_prnt_marital_v2,
                        demo_prnt_ed_v2,
                        interview_age,
                        sex)

In [ ]:
for (i in 1:8) {
    
    Lavaan_SEM[[i]]=resid(lm(Lavaan_SEM[[i]] ~ interview_age + 
                              sex + 
                              factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                              factor(ABCD$demo_comb_income_v2, ordered=F) + 
                              factor(ABCD$demo_ed_v2, ordered=F) +
                              factor(ABCD$demo_prnt_marital_v2, ordered=F), 
                              data=Lavaan_SEM, na.action=na.exclude))
        
    }
        
Lavaan_SEM$z.scores_site[is.na(Lavaan_SEM$z.scores_site)]=mean(Lavaan_SEM$z.scores_site, na.rm=T)   
Lavaan_SEM$upps_y_ss_negative_urgency_site[is.na(Lavaan_SEM$upps_y_ss_negative_urgency_site)]=mean(Lavaan_SEM$upps_y_ss_negative_urgency_site, na.rm=T)        
Lavaan_SEM$upps_y_ss_positive_urgency_site[is.na(Lavaan_SEM$upps_y_ss_positive_urgency_site)]=mean(Lavaan_SEM$upps_y_ss_positive_urgency_site, na.rm=T)        

In [ ]:
ABCD_SEM = '

#regressions

H_site_smri_vol_cdk_total_PC ~ Z_corr_UKB_site
H_site_smri_thick_cdk_mean_PC ~ Z_corr_UKB_site 
H_WM_FA_PC ~ Z_corr_UKB_site 

z.scores_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC 
upps_y_ss_positive_urgency_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC  
upps_y_ss_negative_urgency_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC  

deltaBMI_SD_site ~ z.scores_site + 
        upps_y_ss_positive_urgency_site + upps_y_ss_negative_urgency_site  

#covariance
H_site_smri_vol_cdk_total_PC ~~ H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC
H_site_smri_thick_cdk_mean_PC ~~ H_WM_FA_PC
upps_y_ss_positive_urgency_site ~~ upps_y_ss_negative_urgency_site + z.scores_site
upps_y_ss_negative_urgency_site ~~ z.scores_site


'


ABCD_SEM_model <- sem(ABCD_SEM, data=Lavaan_SEM, missing='listwise', se='robust.sem', estimator='ML')
lavaan:::print.lavaan.summary(summary(ABCD_SEM_model, standardized=TRUE, fit.measures=TRUE))
#fitmeasures(WMHOB_fit) #fit indices

# Calculate follow-up stats - cortical volume + thickness

In [ ]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr.csv', sep=",")

In [ ]:
Stats=matrix(nrow=length(c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr.csv', sep=",")

# Mesulam analysis

In [ ]:
library('effectsize')

In [ ]:
Mesulam=read.table('/dagher/dagher11/filip/Downloads/Mesulam_dkt.csv',sep=';', header=F)
colnames(Mesulam)=c('struct','class')
Mesulam=Mesulam[!(Mesulam$struct=='unknown' | Mesulam$struct=='corpuscallosum'),]
Mesulam
#Mesulam=Mesulam[2:68,]

#1 = paralimbic
#2 = heteromodal
#3 = unimodal
#4 - idiotypic

In [ ]:
volume=cbind(as.data.frame(cbind(ABCD[c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])])))

In [ ]:
Mes_stats=matrix(nrow=4, ncol=5)

index=1

for (i in 1:4) { #
  
    model <- (lm(rowSums(volume[Mesulam$class==i]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20  + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD))
    
    modelperm <- (lmp(rowSums(volume[Mesulam$class==i]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20  + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD, maxIter=10000, center=F))
    
    out=summary(model)
    out2=summary(modelperm)
    eta2_PRS=eta_squared(model)[1,2]
    
    
    name=i
    
    Mes_stats[index,][1]=name#colnames(ABCD[i])
    Mes_stats[index,][2]=eta2_PRS
    Mes_stats[index,][3]=out$coefficients[2,1] # estimate
    Mes_stats[index,][4]=out2$coefficients[2,3] # p-value
    Mes_stats[index,][5]=out$coefficients[2,3] # t-value
       
     
    index=index+1
}

Mes_stats_adj=Mes_stats
Mes_stats_adj[,4]=as.numeric(p.adjust(Mes_stats_adj[,4], method = 'fdr'))
Mes_stats_adj=data.frame(Mes_stats_adj)
colnames(Mes_stats_adj)=c('measure','eta2','estimate','p','t')

Mes_stats_adj[Mes_stats_adj[,4]<0.05,]

In [ ]:
Mes_stats_adj

In [ ]:
thickness=cbind(as.data.frame(cbind(ABCD[c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])])))

In [ ]:
Mes_stats=matrix(nrow=4, ncol=5)

index=1

for (i in 1:4) { #
  
    model <- (lm(rowMeans(thickness[Mesulam$class==i]) ~
                      ABCD$Z_corr_UKB + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD))
    
    modelperm <- (lmp(rowMeans(thickness[Mesulam$class==i]) ~
                      ABCD$Z_corr_UKB + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD, maxIter=10000, center=F))
    
    out=summary(model)
    out2=summary(modelperm)
    eta2_PRS=eta_squared(model)[1,2]
    
    
    name=i
    
    Mes_stats[index,][1]=name#colnames(ABCD[i])
    Mes_stats[index,][2]=eta2_PRS
    Mes_stats[index,][3]=out$coefficients[2,1] # estimate
    Mes_stats[index,][4]=out2$coefficients[2,3] # p-value
    Mes_stats[index,][5]=out$coefficients[2,3] # t-value
       
     
    index=index+1
}

Mes_stats_adj=Mes_stats
Mes_stats_adj[,4]=as.numeric(p.adjust(Mes_stats_adj[,4], method = 'fdr'))
Mes_stats_adj=data.frame(Mes_stats_adj)
colnames(Mes_stats_adj)=c('measure','eta2','estimate','p','t')

Mes_stats_adj

# Other associations

# Positive urgency - volume

In [ ]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_positive_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_positive_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_posurg.csv', sep=",")

# Negative urgency - volume

In [ ]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_negative_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_negative_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_negurg.csv', sep=",")

# Influence of ADI on PRS and BMI SDS

In [ ]:
summary(lmp(ABCD$Z_corr_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                      reshist_addr1_coi_z_ed_nat +
                      reshist_addr1_coi_z_se_nat +
                      reshist_addr1_coi_z_he_nat, data=ABCD, maxIter = 10000, center=F))

summary(lmp(ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                      reshist_addr1_coi_z_ed_nat +
                      reshist_addr1_coi_z_se_nat +
                      reshist_addr1_coi_z_he_nat, data=ABCD, maxIter = 10000, center=F))